In [1]:
df_filter = df['ID'].isin(['A001','C022',...])
df[df_filter]

NameError: name 'df' is not defined

In [ ]:
def Random_Search():
    global best_model, saved_moldel
    
    param_grid = {"n_estimators": range(20, 100, 2),
                  "max_depth": range(4, 50, 2),
                  "min_samples_leaf": range(2, 100, 2),
                  "max_features": sp_randint(1, n_features),
                  "min_samples_split": sp_randint(2, 10),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}

    clf = RandomForestClassifier(class_weight = 'balanced')
    n_iter_search = 500
    estimator = RandomizedSearchCV(clf,
                                   param_distributions = param_grid,
                                   n_iter = n_iter_search,
                                   scoring = 'roc_auc',
                                   verbose = 0,
                                   n_jobs = 1)
        
    fit = estimator.fit(X_train, y_train)

    # Cross validation with 20 iterations to get smoother mean test and train
    # score curves, each time with 20% data randomly selected as a validation set.
    cv_ = ShuffleSplit(n_splits = 20, test_size = 0.20, random_state = 0)
    Plot_learning_curve(estimator, 
                        'Learning Curves',
                        X_train, y_train, 
                        cv = cv_,
                        n_jobs = 1)
     
    Report_scores(estimator.cv_results_, n_top = 3)
    
    best_model = estimator.best_estimator_
    print('\nbest_model:\n', best_model)

    print('\nFeature Importances:', best_model.feature_importances_)
    Plot_predictor_importance(best_model, feature_columns)

    y_predicted = best_model.predict(X_train)
    probabilities = best_model.predict_proba(X_train)

    c_report = classification_report(y_train, y_predicted)
    print('\nClassification report:\n', c_report)

    y_predicted_train = best_model.predict(X_train)
    cm = confusion_matrix(y_train, y_predicted_train)
    auc = roc_auc_score(y_train, y_predicted_train)
    Print_confusion_matrix(cm, auc, 'Confusion matrics of the training dataset')

    y_predicted = best_model.predict(X_test)
    cm = confusion_matrix(y_test, y_predicted)
    auc = roc_auc_score(y_test, y_predicted)

    ntotal = len(y_test)
    correct = y_test == y_predicted
    numCorrect = sum(correct)
    percent = round( (100.0*numCorrect)/ntotal, 6)
    print("\nCorrect classifications on test data: {0:d}/{1:d} {2:8.3f}%".format(numCorrect, ntotal, percent))
    prediction_score = 100.0*best_model.score(X_test, y_test)
    print('Random Forest Prediction Score on test data: %8.3f' % prediction_score)

    model_path = 'C:/sm/BottleRockets/trained_models/sklearn_rf_classify.pkl'
    joblib.dump(best_model, model_path)

    saved_model = joblib.load(model_path)
    y_predicted_test = best_model.predict(X_test)
    cm = confusion_matrix(y_test, y_predicted_test)
    auc = roc_auc_score(y_test, y_predicted_test)
    Print_confusion_matrix(cm, auc, 'Confusion matrics of the test dataset')
    ROC_Curve(best_model, auc)
    Print_Metrics(saved_model)
    return


In [ ]:
def Print_confusion_matrix(cm, auc, heading):
    print('\n', heading)
    print(cm)
    true_negative  = cm[0,0]
    true_positive  = cm[1,1]
    false_negative = cm[1,0]
    false_positive = cm[0,1]
    total = true_negative + true_positive + false_negative + false_positive
    accuracy = (true_positive + true_negative)/total
    precision = (true_positive)/(true_positive + false_positive)
    recall = (true_positive)/(true_positive + false_negative)
    misclassification_rate = (false_positive + false_negative)/total
    F1 = (2*true_positive)/(2*true_positive + false_positive + false_negative)
    print('accuracy.................%7.4f' % accuracy)
    print('precision................%7.4f' % precision)
    print('recall...................%7.4f' % recall)
    print('F1.......................%7.4f' % F1)
    print('auc......................%7.4f' % auc)


In [ ]:

#Plot the learning curves

def Plot_learning_curve(estimator, title, X, y, ylim = None, cv = None,
                        n_jobs = 1, train_sizes = np.linspace(0.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, 
                                                            X, y,
                                                            cv = cv,
                                                            n_jobs = n_jobs,
                                                            train_sizes = train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.show()
    return


In [ ]:
def Print_confusion_matrix(cm, auc, heading):
    print('\n', heading)
    print(cm)
    true_negative  = cm[0,0]
    true_positive  = cm[1,1]
    false_negative = cm[1,0]
    false_positive = cm[0,1]
    total = true_negative + true_positive + false_negative + false_positive
    accuracy = (true_positive + true_negative)/total
    precision = (true_positive)/(true_positive + false_positive)
    recall = (true_positive)/(true_positive + false_negative)
    misclassification_rate = (false_positive + false_negative)/total
    F1 = (2*true_positive)/(2*true_positive + false_positive + false_negative)
    print('accuracy.................%7.4f' % accuracy)
    print('precision................%7.4f' % precision)
    print('recall...................%7.4f' % recall)
    print('F1.......................%7.4f' % F1)
    print('auc......................%7.4f' % auc)


In [ ]:
def Print_Metrics(saved_rf):
    print('\nModel performance on the test data set:')

    # print('Train Accuracy.......', accuracy_score(y_train, best_model.predict(X_train)))
    # print('Validate Accuracy....', accuracy_score(y_valid, best_model.predict(X_valid)))

    y_predict_test  = best_model.predict(X_test)
    mse             = metrics.mean_squared_error(y_test, y_predict_test)
    logloss_test    = metrics.log_loss(y_test, y_predict_test)
    accuracy_test   = metrics.accuracy_score(y_test, y_predict_test)
    accuracy_test2  = best_model.score(X_test, y_test)
    F1_test         = metrics.f1_score(y_test, y_predict_test)
    precision_test  = precision_score(y_test, y_predict_test, average='binary')
    precision_test2 = metrics.precision_score(y_test, y_predict_test)
    recall_test     = recall_score(y_test, y_predict_test, average='binary')
    auc_test        = metrics.roc_auc_score(y_test, y_predict_test)
    r2_test         = metrics.r2_score(y_test, y_predict_test)
   
    #test_auc       = h2o.get_model("best_rf").model_performance(test_data=test).auc()
    #print('Best model performance based on auc: ', test_auc)
    
    header = ["Metric", "Test"]
    table  = [
               ["logloss",   logloss_test],
               ["accuracy",  accuracy_test],
               ["precision", precision_test],
               ["F1",        F1_test],
               ["r2",        r2_test],
               ["AUC",       auc_test]
             ]

    print(tabulate(table, header, tablefmt="fancy_grid"))


In [ ]:
def ROC_Curve(rf, auc):
    one_hot_encoder = OneHotEncoder()
    rf_fit = rf.fit(X_train, y_train)
    fit = one_hot_encoder.fit(rf.apply(X_train))
    y_predicted = rf.predict_proba(X_test)[:, 1]
    false_positive, true_positive, _ = roc_curve(y_test, y_predicted)

    plt.figure()
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(false_positive, true_positive, color='darkorange', label='Random Forest')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve (area = %0.2f)' % auc)
    plt.legend(loc='best')
    plt.show()


In [ ]:
def Correlation_plot(df):
    plt.ioff()
    red_green = ["#ff0000", "#00ff00"]
    sns.set_palette(red_green)
    np.seterr(divide='ignore', invalid='ignore')
    g = sns.pairplot(df,
                     diag_kind = 'kde',
                     hue = 'Altitude',
                     markers = ["o", "D"],
                     size = 1.5,
                     aspect = 1,
                     plot_kws = {"s": 6})
    g.fig.subplots_adjust(right = 0.9)
    plt.show()


In [ ]:
print('Time taken in grid search: {0: .2f}'.format(end - start))

In [ ]:
class Timer:
  def __init__(self):
    self.start = time.time()

  def restart(self):
    self.start = time.time()

  def get_time(self):
    end = time.time()
    m, s = divmod(end - self.start, 60)
    h, m = divmod(m, 60)
    time_str = "%02d:%02d:%02d" % (h, m, s)
    return time_str


In [ ]:
print('Operating system version....', platform.platform())
print("Python version is........... %s.%s.%s" % sys.version_info[:3])
print('scikit-learn version is.....', sklearn.__version__)
print('pandas version is...........', pd.__version__)
print('numpy version is............', np.__version__)
print('matplotlib version is.......', matplotlib.__version__)


In [ ]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics

from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, cohen_kappa_score, make_scorer
from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score
from sklearn.metrics import precision_recall_curve, SCORERS
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.externals import joblib
from operator import itemgetter

In [ ]:
train_data = train_df.copy()
train_data["Age"].fillna(train_df["Age"].median(skipna=True), inplace=True)
train_data["Embarked"].fillna(train_df['Embarked'].value_counts().idxmax(), inplace=True)
train_data.drop('Cabin', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(15,8))
ax = train_df["Age"].hist(bins=15, density=True, stacked=True, color='teal', alpha=0.6)
train_df["Age"].plot(kind='density', color='teal')
ax = train_data["Age"].hist(bins=15, density=True, stacked=True, color='orange', alpha=0.5)
train_data["Age"].plot(kind='density', color='orange')
ax.legend(['Raw Age', 'Adjusted Age'])
ax.set(xlabel='Age')
plt.xlim(-10,85)
plt.show()

In [1]:
def pretty_print_coefs(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)

In [2]:
print ("Ridge model:", pretty_print_coefs(ridge_reg.params_))

NameError: name 'ridge_reg' is not defined

In [ ]:
pd.crosstab(d['agekd_binomial'], d['premarsx_binomial'], normalize='columns')

In [ ]:
conditions = [
    ((d['PREMARSX'] == 1) | (d['PREMARSX'] == 2)),
    ((d['PREMARSX'] == 3) | (d['PREMARSX'] == 4))]
choices = [1,0]
d['premarsx_binomial'] = np.select(conditions, choices, default=np.nan)
d['premarsx_binomial'].value_counts()


conditions = [
    ((dat['relig'] == 1) | (dat['relig'] == 2) | (dat['relig'] == 11)) ,
     (dat['relig'] == 3), 
     (dat['relig'] == 6), 
     (dat['relig'] == 9),
     ((dat['relig'] != 1) & ((dat['relig'] != 2)) & (dat['relig'] != 11) & (dat['relig'] != 3) & 
    (dat['relig'] != 6) & (dat['relig'] != 9))
]
category = ['Christian', 'Jewish', 'Buddhism', 'Muslim', 'others'] #Christian, Jewish, Buddhism, Muslim, and others
dat['relig_new'] = np.select(conditions, category, default=np.nan)

dat.relig_new.value_counts()

In [ ]:
exp = np.exp(result.params)
ci = np.exp(result.conf_int())
ci.columns = ["2.5%", "97.5%"]
ci['exp'] = exp
ci['odds %'] = round((exp - 1), 2)*100
ci

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula(formula2, dat2)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

In [ ]:
agekd_binomial_tab = pd.crosstab(index=d["agekd_binomial"],  # Make a crosstab
                              columns="count")     

def compute_percentage(x):
      pct = float(x/agekd_binomail_tab['count'].sum()) * 100
      return round(pct, 2)

agekd_binomial_tab['%'] = agekd_binomial_tab.apply(compute_percentage, axis=1)

agekd_binomial_tab

In [ ]:
## Cut them into age basket ##
d['agekd_cut'] = pd.cut(d['AGEKDBRN'], bins=[12,20,30,40,50], labels=None, include_lowest= True)
print (pd.value_counts(d["agekd_cut"], sort=True))

In [ ]:
bb = d[['AGEKDBRN','ID']].groupby('AGEKDBRN').count()
bb['%'] = bb/bb.ID.sum()*100
bb = bb.rename(columns = {'ID' : 'Total'})

In [ ]:
res = pd.crosstab(d.SIBS, d.sib_cut)
res.astype('float').div(res.sum(axis=0), axis=1)

In [ ]:
sib_cuts.sort_values(ascending = False)

In [ ]:
dat.filter(like='age').columns

aa = dat.filter(like = "age").columns

for x in aa: 
    print(dat[x].describe())

In [ ]:
wvs[['manner','sex','ses','immigrant','education','live with parents','privacy','marital', 'age']].head() 

In [ ]:
wvs['manner']=7-(wvs['manner']) ## reverse respond : higher variable means more mannered ... ##

In [ ]:
wvssub = pd.concat([wvs_new.manner, wvs_new.age, wvs_new.citizen, wvs_new.education, wvs_new.parenting, wvs_new.secret_survey, wvs_new.female, wvs_new.married, wvs_new.V2, 
                   pd.get_dummies(wvs_new.ses.astype(int), prefix='ses')],  axis = 1 )
wvssub.columns
 

In [ ]:
lm2 = smf.ols(formula = f, 
              data = wvssub, 
              subset = (wvssub['V2']==840)).fit()

In [ ]:
lm_ln = smf.ols(formula = "ln_agewed ~ ln_realrinc", data = dat).fit(cov_type='HC3')

In [ ]:
# partial F-test
sm.stats.anova_lm(lm1, lm3)

In [ ]:
# Let's try dummy variable this time
lm1_dummy = smf.ols(formula = 'Neighbors ~ C(Robberies) + C(Alcohol_consumption) + C(Police_deprive) + C(Racist) + C(Drug_sale)', data = wvs_new, subset= (wvs_new['Country'] == 840)).fit()
print (lm1_dummy.summary())

In [ ]:
plt.plot(sub_dat["age"], lm_interaction.params[0] + lm_interaction.params[1] * 1 + lm_interaction.params[2] * sub_dat["age"] + lm_interaction.params[3] * 1 * sub_dat["age"], 'b', label = 'Egypt', alpha = 0.9)
plt.plot(sub_dat["age"], lm_interaction.params[0] + lm_interaction.params[1] * 0 + lm_interaction.params[2] * sub_dat["age"] + lm_interaction.params[3] * 0 * sub_dat["age"], 'r', label = 'America', alpha = 0.9)
plt.title("Reading a newspaper, America (red) vs. Egypt (blue)")
plt.xlabel("age")
plt.ylabel("Reading newspaper")
plt.show()

In [ ]:
plt.figure()
plt.hist(degree_lm.resid, color = "skyblue", ec="black")
plt.xlabel("Residuals")
plt.ylabel("Density")
plt.show()

In [ ]:
degree_glm = smf.glm(formula=formula, data=sub, family = sm.families.Binomial())
result = degree_glm.fit()
print(result.summary())


degree_logit = sm.formula.logit(formula = formula, data = sub).fit()
print (degree_logit.summary()) 

In [ ]:
def logit2prob (logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds) 
    return(prob);


intercept = degree_logit.params.Intercept
b_joblose = degree_logit.params.joblose
b_jobfind = degree_logit.params.jobfind
b_satjob = degree_logit.params.satjob
b_satfin = degree_logit.params.satfin
b_finalter = degree_logit.params.finalter
b_income06 = degree_logit.params.income06
b_class_1 = degree_logit.params.class_1

In [ ]:
# Person A's predicted prob model:

'''
A does not think that he/she will lose his/her job (4),
-- A thought that finding a job was very easy (3),
-- A is very satisfied with the current position (4) and financial status (4),
-- A thinks that his/her financial status is improving (3),
-- A's income falls at the top tier (25),
-- A's social class is the upper class (4)
'''

logits_degree1 = intercept + (4 * b_joblose) + (3 * b_jobfind) + (4 * b_satjob) + (3 * b_satfin) + (3 * b_finalter) + (25 * b_income06) + (4 * b_class_1)
logit2prob(logits_degree1)

In [ ]:
abrape_tab = pd.crosstab(index = dat['abrape'], columns = 'count')
abrape_tab['%'] = 100 * abrape_tab / abrape_tab.sum()
abrape_tab['cum_%'] = 100 * abrape_tab['count'].cumsum()/abrape_tab['count'].sum()
abrape_tab

In [ ]:
dat.loc[dat['educ'] == 98] = np.nan
dat.loc[dat['educ'] == 99] = np.nan
dat = dat.dropna(subset = ['educ'])
plt.hist(dat.educ, bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85, align = 'mid')
plt.title("Distribution of Education")
plt.show()

In [ ]:
## plot out the realinc distribtuion 
dat = dat.dropna(subset = ['realinc'])
import seaborn as sns
sns.distplot(dat.realinc).set_title("Distribution of family real income")